### The purpose of this python notebook is to retrieve the hydraulic conductivity and retention capacity for the soils in the dataset using the staringreeks from Wageningen Environmental Research (WER).

Required inputs:
- Tabular dataset of soil sample points, with data for lutum fraction, soil organic carbon, and soil type classification for each point.
- Kriged raster map of groundwater depths in the area relative to sea level (NAP).
- A digital terrain model height map of the area. 

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio as rio
from rasterio.plot import show

In [2]:
# Load the datasets

topsoil_data = gpd.read_file('./Data/topsoil_samples.gpkg')
subsoil_data = gpd.read_file('./Data/subsoil_samples.gpkg')
groundwater_raster = rio.open('./Data/kriged_groundwater.tif')
heightmap = rio.open('./Data/ahn_ribu.tif')
staringreeks_df = pd.read_csv('./Data/staringreeks_2018.csv')

In [6]:
staringreeks_df.head()


,year,unit,name,wcr,wcs,alpha,npar,lambda,ksfit
0,2018,1,B01,0.02,0.427,0.022,1.735,0.981,31.225
1,2018,2,B02,0.02,0.434,0.022,1.349,7.202,83.242
2,2018,3,B03,0.02,0.443,0.015,1.505,0.139,19.077
3,2018,4,B04,0.02,0.462,0.015,1.397,0.295,34.884
4,2018,5,B05,0.01,0.381,0.043,1.808,0.024,63.650


In [ ]:
topsoil_data.head()

,datum,boorpunt_ID,BOVENKANT,ONDERKANT,analysemonster_ID,lutumgehalte,organisch_stofgehalte,leem,type,geometry
0,NaT,12088,0.1,0.5,8616,1.0,0.8,22.8,B03,POINT (121641 483535)
1,NaT,17290,0.0,0.5,12928,1.0,0.7,22.2,B03,POINT (122317 483576)
2,NaT,17291,0.0,0.5,12929,5.0,11.9,89.4,B05,POINT (122284 483579)
3,NaT,17357,0.0,0.5,12973,1.0,0.4,20.4,B03,POINT (121484 483339)
4,NaT,17365,0.0,0.2,12979,1.5,16.9,119.4,B15,POINT (122391.906 484589.127)


In [7]:
# For each point in the soil data, find the staringreeks entry that matches the soil type, then append the data to the soil data.
topsoil_data.merge(
    staringreeks_df,
    left_on='type',
    right_on='name',
    how='left',
    suffixes=('', '_staringreeks')
)

,datum,boorpunt_ID,BOVENKANT,ONDERKANT,analysemonster_ID,lutumgehalte,organisch_stofgehalte,leem,type,geometry,year,unit,name,wcr,wcs,alpha,npar,lambda,ksfit
0,NaT,12088,0.10,0.5,8616,1.0,0.8,22.8,B03,POINT (121641 483535),2018.0,3.0,B03,0.02,0.443,0.015,1.505,0.139,19.077
1,NaT,17290,0.00,0.5,12928,1.0,0.7,22.2,B03,POINT (122317 483576),2018.0,3.0,B03,0.02,0.443,0.015,1.505,0.139,19.077
2,NaT,17291,0.00,0.5,12929,5.0,11.9,89.4,B05,POINT (122284 483579),2018.0,5.0,B05,0.01,0.381,0.043,1.808,0.024,63.650
3,NaT,17357,0.00,0.5,12973,1.0,0.4,20.4,B03,POINT (121484 483339),2018.0,3.0,B03,0.02,0.443,0.015,1.505,0.139,19.077
4,NaT,17365,0.00,0.2,12979,1.5,16.9,119.4,B15,POINT (122391.906 484589.127),2018.0,15.0,B15,0.01,0.528,0.024,1.282,-1.478,87.451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1329,2024-10-08,726799,0.08,0.5,1816268,1.0,1.0,24.0,B03,POINT (122361.323 484166.112),2018.0,3.0,B03,0.02,0.443,0.015,1.505,0.139,19.077
1330,2024-10-08,726802,0.04,0.5,1816269,1.0,1.0,24.0,B03,POINT (122310.037 484251.693),2018.0,3.0,B03,0.02,0.443,0.015,1.505,0.139,19.077
1331,2024-04-23,743330,0.00,0.5,1834316,2.0,3.3,37.8,B04,POINT (121365.222 484060.926),2018.0,4.0,B04,0.02,0.462,0.015,1.397,0.295,34.884
1332,2023-08-29,745259,0.05,0.5,1836047,5.6,0.6,21.6,B03,POINT (122197.889 484241.141),2018.0,3.0,B03,0.02,0.443,0.015,1.505,0.139,19.077
